## TensorFlow 多线程输入数据处理框架

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 7.3 多线程输入数据处理框架

在一个程序中，这些独立运行的程序片断叫作“线程”（Thread），利用它编程的概念就叫作“多线程处理”。

复杂的预处理过程也会减慢整个训练过程。为了避免图像预处理成为神经网络模型训练效率的瓶颈，TensorFlow提供了一套多线程处理输入数据的框架。  
经典输入数据处理流程：  
① 指定原始数据的文件列表  
② 创建文件列表队列  
③ 从文件中读取数据  
④ 数据预处理  
⑤ 整理成Batch作为神经网络输入

对于变量，可以通过赋值操作修改变量的取值；对于队列，修改状态的操作主要有Enqueue、EnqueueMany和Dequeue：  

In [1]:
import tensorflow as tf

# 创建一个先进先出队列，指定队列中最多可以保存两个元素，并且指定为整形
q = tf.FIFOQueue(2, "int32")

# 使用enqueue_many函数来初始化队列中的元素，
# 和变量初始化类似，使用队列前需要明确调用这个初始化过程
init = q.enqueue_many(([0, 10],))

# 使用dequeue函数将队列中第一个元素出队列，这个元素的值将被保存在x中
x = q.dequeue()
# 出队元素加1
y = x + 1
# 将加1后的值重新加入队列
q_inc = q.enqueue([y])

with tf.Session() as sess:
    # 队列初始化操作
    init.run()
    for _ in range(5):
        # 运行q_inc将执行数据出队列、出队的元素加1、重新加入队列这个过程
        v, _ = sess.run([x, q_inc])
        # 打印出队元素的值
        # 队列开始有［0, 10］两个元素，第一个出队的为0 ，加1之后再次入队得到的队列为［10, 1］：
        # 第二次出队的为10，加1之后入队的为11，得到的队列为［1, 11］；以此类推.
        print(v)

0
10
1
11
2


TensorFlow中两种队列：FIFOQueue（如上）、RandomShuffleQueue  
后者会将队列中的元素打乱，每次出队操作得到的是从当前队列所有元素中随机选择的一个。

在TensorFlow中，队列不仅仅是一种数据结构，还是**异步计算张量取值**的一个重要机制。比如多个线程可以同时向一个队列中写元素，或者同时读取一个队列中的元素。  
TensorFlow提供了tf.Coordinator和tf.QueueRunner两个类来完成**多线程协同**的功能:

● **tf.Coordinator**  
主要用于协同多个线程一起停止，并提供了should_stop、request_ stop和join三个函数。在启动线程之前，需要先声明一个tf.Coordinator类，并将这个类传入每一个创建的线程中。启动的线程需要一直查询tf.Coordinator类中提供的should_stop函数，当这个函数的返回值为True时，则当前线程也需要退出。每一个启动的线程都可以通过调用request_stop函数来通知其他线程退出。当某一个线程调用request_stop函数之后，should_stop函数的返回值将被设置为True这样其他的线程就可以同时终止了。  
以下程序展示了如何使用tf.Coordinator:

In [22]:
import numpy as np
import threading
import time


# 线程中运行的程序，这个程序每隔1秒判断是否需要停止并打印自己的ID
def MyLoop(coord, worker_id):
    # 使用tf.Coordinator类提供的协同工具判断当前线程是否需要停止
    while not coord.should_stop():
        # 随机停止所有线程
        if np.random.rand() < 0.1:
            print("Stoping from id: %d\n" % worker_id),
            # 调用coord.request_stop函数来通知其他所有线程停止
            coord.request_stop()
        else:
            print("Working on id: %d\n" % worker_id),
        time.sleep(1)
        
# 声明一个tf.train.Coordinator类来协同多个线程
coord = tf.train.Coordinator()
# 声明创建5个线程
threads = [threading.Thread(target=MyLoop, args=(coord, i,)) for i in range(5)]
# 启动所有线程
for t in threads:t.start()
# 等待所有线程停止
coord.join(threads)

# 当所有线程启动之后，每个线程会打印各自的ID，于是前面4行打印出了它们的ID。
# 然后在暂停1秒之后，所有线程又开始第二遍打印ID。在这个时候有一个线程退出的条件达到，
# 于是调用了coord.request_stop函数来停止所有其他的线程。然而在打印'Stoping from id: 1'之后，
# 可以看到有线程仍然在输出这是因为这些线程已经执行完coord.should_stop的判断，
# 于是仍然会继续输出自己的ID，但在下一轮判断是否需要停止时将退出线程。

Working on id: 0
Working on id: 1

Working on id: 2


Working on id: 3
Working on id: 4


Working on id: 0

Stoping from id: 1
Working on id: 2




● **tf.QueueRunner**  
主要用于启动多个线程来操作同一个队列，启动的这些线程可以通过上面介绍的tf.Coordinator类来统一管理。  
以下代码展示使用tf.QueueRunner和tf.Coordinator来管理多线程队列操作：

In [24]:
# 先声明一个先进先出的队列，队列中最多100个元素，类型为实数型
queue = tf.FIFOQueue(100, "float")
# 定义队列的入队操作
enqueue_op = queue.enqueue([tf.random_normal([1])])

# 使用tf.train.QueueRunner来创建多个线程运行队列的入队操作
#   第一个参数给出了被操作的队列；
#   第二个参数表示需要启动5个线程，每个线程中运行的是enqueue_op操作
qr = tf.train.QueueRunner(queue, [enqueue_op] * 5)

# 将定义过的QueueRunner加入TensorFlow计算图上指定的集合，默认tf.GraphKeys,QUEUE_RUNNERS
tf.train.add_queue_runner(qr)
# 定义出队操作
out_tensor = queue.dequeue()

with tf.Session() as sess:
    # 使用tf.train.Coordinator来协同启动的线程
    coord = tf.train.Coordinator()
    
    # 使用tf.train.QueueRunner时，需要明确调用tf.train.start_queue_runners来启动所有线程，
    # 否则因为没有线程运行入队操作，当调用出队操作时，程序会一直等待入队操作被运行。
    # tf.train.start_queue_runners函数会默认启动tf.GraphKeys.QUEUE_RUNNERS集合中所有的QueueRunner，
    # 因为这个函数只支持启动指定集合中的QueueRnner，所以一般来说
    # tf.train.add_queue_runner函数和tf.train.start_queue_runners函数会指定同一个集合。
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # 获取队列中的值
    for _ in range(3):
        print(sess.run(out_tensor)[0])
    
    # 使用tf.train.Coordinator来停止所有线程
    coord.request_stop()
    coord.join(threads)
    
# 以上程序将启动5个线程来执行队列入队的操作，其中每一个线程都是将随机数写入队列。
# 于是在每次运行出队操作时，可以得到一个随机数

0.63297355
-1.3789966
-1.007176


虽然一个TFRecord文件中可以存储多个训练样例，但是当训练数据较大时，可以将数据分成多个TFRecord文件来提高处理效率。  
TensorFlow提供了**tf.train.match_filenames_once**来获取符合一个正则表达式的所有文件列表，得到的文件列表可以通过tf.train.string_input_producer函数进行有效的管理：  
**tf.train.string_input_producer**函数会将初始化时提供的文件列表创建一个输入队列，输入队列中原始的元素为文件列表中的所有文件；  
● 通过设置**shuffle**参数，tf.train.string_input_producer函数支持随机打乱文件列表中的文件出队顺序，随机打乱文件顺序以及加入输入队列的过程会跑在一个单独的线程上，这样不会影响获取文件的速度；  
● tf.train.string_input_producer生成的输入队列可以同时被多个文件读取线程操作，而且输入队列会将队列中的文件均匀地分给不同的线程，不出现有些文件被处理过多次而有些文件还没有被处理过的情况；  
● 通过设置**num_epoches**参数，tf.train.string_input_producer可以限制加载初始文件的最大轮数，所有文件已经被使用了设定的轮数后如果继续尝试读取会报OutOfRange错误，神经网络测试时可设为1。   
先给出一个简单的程序来生成样例数据：

In [25]:
import tensorflow as tf

# 创建TFRecord文件的辅助函数
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 模拟海量数据情况下将数据写入不同的TFRecord文件
num_shards = 2               # 定义总共写入多少个文件
instances_per_shard = 2      # 定义每个文件中有多少个数据
for i in range(num_shards):
    # 将数据分为多个文件时，可以将不同文件以类似OOOOn-of-OOOOm的后缀区分。
    # 其中m表示了数据总共被存在了多少个文件中，n表示当前文件的编号。
    # 这样既方便了通过正则方式获取文件列表，又在文件名中加入了更多的信息。
    filename = ('data.tfrecords-%.5d-of-%.5d' % (i, num_shards)) 
    
    # 将Example结构写入TFRecord文件
    writer = tf.python_io.TFRecordWriter(filename)
    for j in range(instances_per_shard):
    # Example结构仅包含当前样例属于第几个文件以及是当前文件的第几个样本。
        example = tf.train.Example(features=tf.train.Features(feature={
            'i': _int64_feature(i),
            'j': _int64_feature(j)}))
        writer.write(example.SerializeToString())
    writer.close()

程序运行之后，在指定的目录下将生成两个文件：指定路径下的data.tfrecords-00000-of-00002 和 data.tfrecords-00001-of-00002。每一个文件中存储了两个样例。  
以下代码展示了tf.train.match_filenames_once函数和tf.train.string_input_producer函数的使用方法：

In [26]:
# 使用tf.train.match_filenames_once函数获取文件列表
files = tf.train.match_filenames_once("data.tfrecords-*")

# 使用tf.train.string_input_producer函数创建输入队列
# 这里shuffle设为False来避免随机打乱读文件的顺序，真实问题一般True
filename_queue = tf.train.string_input_producer(files, shuffle=False)

# 如7.1节中读取并解析一个样本
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
      serialized_example,
      features={
          'i': tf.FixedLenFeature([], tf.int64),
          'j': tf.FixedLenFeature([], tf.int64),
      })


with tf.Session() as sess:
    # 虽然本段程序中没有声明任何变量，但是使用tf.train.初始化一些变量
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    
    # 查看文件名列表
    print(sess.run(files))
    
    # 声明tf.train.Coordinator类来协同不同的线程，并启动线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 多次执行获取数据的操作
    for i in range(6):
        print(sess.run([features['i'], features['j']]))
    coord.request_stop()
    coord.join(threads)

[b'.\\data.tfrecords-00000-of-00002' b'.\\data.tfrecords-00001-of-00002']
[0, 0]
[0, 1]
[1, 0]
[1, 1]
[0, 0]
[0, 1]


TensorFlow提供了**tf.train.batch**和**tf.train.shuffle_batch**函数来将单个的样例组织成batch的形式输出。这两个函数都会生成一个队列，队列的入队操作是生成单个样例的方法，而每次出队得到的是一个batch的样例，它们唯一的区别在于是否会将数据顺序打乱。  
以下代码展示了tf.train.batch的使用方法：

In [27]:
# 使用7.3.2中方法读取并解析得到样例，
# 这里假设Example中i表示样例特征向量，如图像像素矩阵，y表示对应标签
example, label = features['i'], features['j']

# 一个batch中样例个数
batch_size = 3
# 组合样例的队列中最多可以存储的样例个数。这个队列如果太大，那么需要占用很多内在资源：
# 如果太小，那么出队操作可能会因为没有数据而被阻碍（block），从而导致训练效率降低。
# 一般来说这个队列的大小会和每一个batch的大小相关，下面给出了设置队列大小的一种方式。
capacity = 1000 + 3 * batch_size

# 使用tf.train.batch 函数来组合样例：
#    ［example,label］参数给出了需要组合的元素，
#         一般example和label分别代表训练样本和这个样本对应的正确标签。
#     batch_size参数给出了每个batch 中样例的个数。
#     capacity给出了队列的最大容量。
#         当队列长度等于容量时，TensorFlow将暂停入队操作，而只是等待元索出队；
#         当元素个数小于容量时，TensorFlow将自动重新启动入队操作。
example_batch, label_batch = tf.train.batch([example, label], 
                                            batch_size=batch_size, 
                                            capacity=capacity)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 获取并打印组合之后的样例。真实问题中，这个输出一般会作为神经网络的输入
    for i in range(3):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
        
    coord.request_stop()
    coord.join(threads)
    
# 从这个输出可以看到tf.train.batch 函数可以将单个的数据组织成3个一组的batch,
# Eexample, label 中读到的数据依次为：
# example: 0, lable: 0
# example: 0, lable: 1
# example: 1, lable: 0
# example: 1, lable: 0
# 这是因为tf.train.batch函数不会随机打乱顺序，所以组合之后得到的数据组合成了上面给出的输出。

[0 0 1] [0 1 0]
[1 0 0] [1 0 1]
[1 1 0] [0 1 0]


下面展示了tf,train.shuffle_batch函数的使用方式：

In [29]:
# 和tf.train.batch的样例代码一样产生example和label。
example, label = features ['i'], features ['j']

batch_size = 3
capacity = 1000 + 3 * batch_size

# 使用tf.train.shuffle_batch函数来组合样例。
# tf.train.shuffle_batch函数的参数大部分和tf.train.batch函数相似，
# 但是min_after_dequeue参数是tf.train.shuffle_batch函数特有的。
# min_after_dequeue 参数限制了出队时队列中元素的最少个数，
# 因为当队列中元素太少时，随机打乱样例顺序的作用就不太了。
# 当出队函数被调用但是队列中元素个数不够时，出队操作将等待里多的元素入队才会完成。
# 如果min_after_dequeue参数被设定，capacity也应该相应调整来满足性能需求。
example_batch, label_batch = tf.train.shuffle_batch(
    [example, label], batch_size=batch_size,
    capacity=capacity, min_after_dequeue=30)

# 和tf.train.batch的样例代码一样打印example_batch, label_batch。
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 获取并打印组合之后的样例。真实问题中，这个输出一般会作为神经网络的输入
    for i in range(3):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
        
    coord.request_stop()
    coord.join(threads)
    
# 从输出可以看到，得到的样例顺序已经被打乱了。

[0 1 1] [1 0 1]
[1 0 0] [1 0 0]
[0 1 0] [1 1 0]


tf.train.batch函数和tf.train.shuffle_batch函数除了可以将单个训练数据整理成输入batch，也提供了**并行化处理输入数据**的方法。

如果需要多个线程处理不同文件中的样例时，可以使用tf.train.shuffle_batch_join函数，此函数会从输入文件队列中获取不同的文件分配给不同的线程。  
优劣比较：  
● tf.train.shuffle_batch 函数，不同线程会读取同一个文件。如果一个文件中的样例比较相似（比如都属于同一个类别），那么神经网络的训练效果有可能会受到影响。所以在使用tf.train.shuffle_batch函数时，需要尽量将同一个TFRecord 文件中的样例随机打乱。  
● tf.train.shuffle_batch_join函数，不同线程会读取不同文件。如果读取数据的线程数比总文件数还大，那么多个线程可能会读取同一个文件中相近部分的数据。而且多个线程读取多个文件可能导致过多的硬盘寻址，从而使得读取效率降低。

回顾**经典输入数据处理流程**：  
① 指定原始数据的文件列表  
② 创建文件列表队列  
③ 从文件中读取数据  
④ 数据预处理  
⑤ 整理成Batch作为神经网络输入  
下面代码给出完成该流程的程序（**非常重要！！！**）：

In [30]:
import tensorflow as tf

# 1. 创建文件列表，并通过文件列表创建输入文件队列。在调用输入数据处理流程前，需要统一
# 所有原始数据的格式并将它们存储到TFRecord文件中。下面给出的文件列表中应该包含所有
# 提供训练数据的TFRecord 文件。
files = tf.train.match_filenames_once("output.tfrecords")
filename_queue = tf.train.string_input_producer(files, shuffle=False) 


# 2. 解析TFRecord文件里的数据。这里假设image_raw中存储的是图像
# 的原始数据，pixels代表图片的像素数，label为该样例所对应的标签。
reader = tf.TFRecordReader()
_,serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
    features={
        'image_raw':tf.FixedLenFeature([],tf.string),
        'pixels':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })

# 将原始图像数据解析出像素矩阵
decoded_images = tf.decode_raw(features['image_raw'], tf.uint8)
retyped_images = tf.cast(decoded_images, tf.float32)
images = tf.reshape(retyped_images, [784])
labels = tf.cast(features['label'], tf.int32)
#pixels = tf.cast(features['pixels'], tf.int32)

# 3. 将文件以100个为一组打包
min_after_dequeue = 10000
batch_size = 100
capacity = min_after_dequeue + 3 * batch_size

image_batch, label_batch = tf.train.shuffle_batch([images, labels], 
                                                    batch_size=batch_size, 
                                                    capacity=capacity, 
                                                    min_after_dequeue=min_after_dequeue)


# 4. 建立神经网络模型，训练
# 定义模型结构
def inference(input_tensor, weights1, biases1, weights2, biases2):
    layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
    return tf.matmul(layer1, weights2) + biases2

# 模型相关的参数
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500
REGULARAZTION_RATE = 0.0001   
TRAINING_STEPS = 5000        

weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))

weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

y = inference(image_batch, weights1, biases1, weights2, biases2)
    
# 计算交叉熵及其平均值
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=label_batch)
cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
# 损失函数的计算
regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
regularaztion = regularizer(weights1) + regularizer(weights2)
loss = cross_entropy_mean + regularaztion

# 优化损失函数
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
# 初始化会话，并开始训练过程。
with tf.Session() as sess:
    # tf.global_variables_initializer().run()
    sess.run((tf.global_variables_initializer(),
              tf.local_variables_initializer()))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # 循环的训练神经网络。
    for i in range(TRAINING_STEPS):
        if i % 1000 == 0:
            print("After %d training step(s), loss is %g " % (i, sess.run(loss)))
                  
        sess.run(train_step)
        
    # 停止所有线程
    coord.request_stop()
    coord.join(threads)

After 0 training step(s), loss is 330.318 
After 1000 training step(s), loss is 1.98627 
After 2000 training step(s), loss is 1.75031 
After 3000 training step(s), loss is 1.33348 
After 4000 training step(s), loss is 1.06709 


可以看到速度明显加快！

总结：  
输入数据处理的第一步为通过tf.train.match_filenames_once获取存储训练数据的文件列表，假如这个文件列表为{A,B,C}。    
然后通过tf.train.string_input_producer函数，可以选择性地将文件列表中文件的顺序打乱，并加入输入队列（因为是否打乱文件的顺序是可选的，所以在下中通过虚线表示）。tf.train.string_input_producer函数会生成并维护一个输入文件队列，不同线程中的文件读取函数可以共享这个输入文件队列。  
在读取样例数据之后，需要将图像进行预处理。图像预处理的过程也会通过tf.train.shuffle_batch提供的机制并行地跑在多个线程中。输入数据处理流程的最后通过tf.train.shuffle_batch函数将处理好的单个输入样例整理成batch提供给神经网络的输入层。  
通过这种方式，可以有效地提高数据预处理的效率，避免数据预处理成为神经网络模型训练过程中的性能瓶颈。